In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

In [34]:
path = '../Data/Preprocessed/rcrwireless2.csv'

df_origin = pd.read_csv(path)
df_origin[['Keywords', 'Category', 'Preprocessed', 'topicsRatio']] = df_origin[['Keywords', 'Category', 'Preprocessed', 'topicsRatio']].applymap(eval)
df_origin['Date'] = df_origin['Date'].apply(lambda x : pd.to_datetime(x.split('-')[0]))
df_origin = df_origin.drop(columns = ['Title__stop', 'SummarizedText__stop', 'PreprocessedTitle', 'PreprossedSummarizedText'])
df_origin['year'] = df_origin['Date'].dt.year
df_origin['month'] = df_origin['Date'].dt.month

In [12]:
df_origin.head(2)

,Title,Author,Date,Text,Keywords,Href,Category,SummarizedText,Preprocessed,propTopic,propTopicRatio,topicsRatio
0,"next g alliance, india’s bharat 6g alliance in...",Juan Pedro Tomás,2023-01-01,atis’ next g alliance (nga) and india’s bhara...,"[6G, Featured, Networks]",https://www.rcrwireless.com/20230911/6g/nextg-...,[6G],thing to increase efficiency.,"[next, ind, bhar, 6g, ink, coop, deal, increas...",5,0.8238,"[(0, 0.11615509), (1, 0.01203243), (2, 0.01199..."
1,south korea to start project to develop future...,Juan Pedro Tomás,2023-01-01,the government of south korea expects to start...,"[6G, Featured, Spectrum, Standards]",https://www.rcrwireless.com/20230824/6g/south-...,[6G],thing of 6G systems,"[sou, kore, start, project, develop, fut, 6g, ...",5,0.6849,"[(0, 0.26730815), (5, 0.68486685)]"


In [13]:
# propTopic별 비율

df = df_origin.copy()

df.groupby(['propTopic'])['Href'].nunique().reset_index()

,propTopic,Href
0,0,896
1,1,559
2,2,400
3,3,402
4,4,272
5,5,1131
6,6,810


In [27]:
# propTopic 및 Category별 비율
df = df_origin.copy()
result_df = pd.DataFrame()

for column in ['6G', '6E', 'MIMO', 'Automotive']:
    condition = df['Category'].apply(lambda x : column in x)
    tmp = df[condition]
    tmp = tmp.groupby(['propTopic'])['Href'].nunique().reset_index()
    tmp['category'] = column
    result_df = pd.concat([result_df, tmp], axis = 0)

In [28]:
result_df = result_df.pivot_table(index = ['category'], columns = ['propTopic'], values = ['Href']).fillna(0)
result_df = round(result_df / result_df.sum().sum() * 100, 2)
result_df

Href                                     
propTopic       0     1     2     3     4      5     6
category                                              
6E           2.33  2.60  1.31  1.48  0.71   2.33  1.79
6G           2.54  2.19  1.02  1.25  0.73   7.37  2.60
Automotive  12.26  4.21  3.04  4.83  3.62   4.02  8.95
MIMO         2.39  4.04  3.56  1.19  0.79  11.47  5.37

In [30]:
result_df.loc['sum'] = result_df.sum(axis = 0)

In [31]:
result_df

Href                                       
propTopic       0      1     2     3     4      5      6
category                                                
6E           2.33   2.60  1.31  1.48  0.71   2.33   1.79
6G           2.54   2.19  1.02  1.25  0.73   7.37   2.60
Automotive  12.26   4.21  3.04  4.83  3.62   4.02   8.95
MIMO         2.39   4.04  3.56  1.19  0.79  11.47   5.37
sum         19.52  13.04  8.93  8.75  5.85  25.19  18.71

In [40]:
# topic별 비율

df = df_origin.copy()
df.groupby(['year', 'propTopic'])['Href'].nunique().reset_index().pivot_table(index = 'year', columns = 'propTopic', values = 'Href').fillna(0)

propTopic,0,1,2,3,4,5,6
year,,,,,,,
1999,24.0,89.0,47.0,224.0,67.0,112.0,54.0
2008,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010,2.0,5.0,0.0,5.0,7.0,5.0,2.0
2011,4.0,11.0,7.0,10.0,8.0,26.0,14.0
2012,10.0,7.0,7.0,13.0,4.0,13.0,11.0
2013,7.0,15.0,23.0,13.0,7.0,16.0,37.0
2014,22.0,24.0,22.0,10.0,4.0,16.0,47.0
2015,31.0,20.0,31.0,9.0,7.0,30.0,78.0
2016,88.0,45.0,38.0,6.0,19.0,66.0,83.0


In [42]:
df.head(2)

,Title,Author,Date,Text,Keywords,Href,Category,SummarizedText,Preprocessed,propTopic,propTopicRatio,topicsRatio,year,month
0,"next g alliance, india’s bharat 6g alliance in...",Juan Pedro Tomás,2023-01-01,atis’ next g alliance (nga) and india’s bhara...,"[6G, Featured, Networks]",https://www.rcrwireless.com/20230911/6g/nextg-...,[6G],thing to increase efficiency.,"[next, ind, bhar, 6g, ink, coop, deal, increas...",5,0.8238,"[(0, 0.11615509), (1, 0.01203243), (2, 0.01199...",2023,1
1,south korea to start project to develop future...,Juan Pedro Tomás,2023-01-01,the government of south korea expects to start...,"[6G, Featured, Spectrum, Standards]",https://www.rcrwireless.com/20230824/6g/south-...,[6G],thing of 6G systems,"[sou, kore, start, project, develop, fut, 6g, ...",5,0.6849,"[(0, 0.26730815), (5, 0.68486685)]",2023,1
